In [127]:
"""
This could be markdown later. Include something about the note book in here if necessary.
"""

'\nThis could be markdown later. Include something about the note book in here if necessary.\n'

In [359]:
#Import libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pendulum
import datetime

In [118]:
"""
Data Wrangling Section 1: Assessment Page Views.csv
"""
#Read in csv as dataframe
#Using a smaller subset of the data since testing with 6mil+ rows would be too intensive
df = pd.read_csv('/Users/ethanrowe/Dev/springboard_data/KSU Assessment Page Views Fall 2018.csv', nrows = 10000)

In [48]:
#Eliminate Outliers using the 1.5*interquartile rule
IQR = float(df['duration'].quantile([0.75])) - float(df['duration'].quantile([0.25]))
upper_bound = float(df['duration'].quantile([0.75])) + 1.5*IQR
lower_bound = float(df['duration'].quantile([0.25])) - 1.5*IQR

df_clean = df[(df['duration'] < upper_bound) & (df['duration'] > lower_bound)]

In [376]:
print("Cleaned Data:")
print(df_clean.shape)
print(df.isna().sum())
#df_clean.to_csv('KSU Assessment Page Views Fall 2018 Clean.csv')

Cleaned Data:
(8846, 13)
assessment_attempt_id                                         0
assessment_type                                               0
canvas_assignment_id                                          0
canvas_course_id                                              0
canvas_section_id                                             0
created_at                                                    0
duration                                                      0
id                                                            0
load_time                                                     0
page_id                                                       0
raw_lti_param_ext_outcome_submission_submitted_at_accepted    0
submit_time                                                   0
user_param_external_user_id                                   0
dtype: int64


In [110]:
#Group by page and by students to compare average assignment durations
assignment_df = pd.DataFrame(df_clean.groupby(['canvas_assignment_id'])['duration'].mean())
print('\nGrouped by Assignment')
assignment_df.columns = ['Average Duration']
print(page_df.shape)

user_df = pd.DataFrame(df_clean.groupby(['user_param_external_user_id', 'canvas_assignment_id'])['duration'].mean())
print('\nGrouped by User')
user_df.columns = ['Average Duration']
print(user_df.shape)


#page_df.to_csv('KSU Assessment Assignment Duration Averages.csv')
#user_df.to_csv('KSU Assessment Student Views by User.csv')    


Grouped by Assignment
(580, 1)

Grouped by User
(1865, 1)


In [408]:
#User_df is good for visualizing the data, but a df that is easier to work with will be labeled sub_df. It has the 
#same columns as user_df, just in a different order with a numerical index. It will also be sorted by 
#canvas_assignment_id in order to match assignment_df's index
sub_df = user_df.reset_index()  
sub_df = sub_df.set_index('canvas_assignment_id')  
sub_df = sub_df.sort_index()     
sub_df = sub_df.reset_index()    

#Using the user_df, turn it back into a dictionary grouped by canvas_assignment ids
dfs = {}       
for entry in sub_df.iterrows():  
    if index != entry[1]['canvas_assignment_id']:  
        index = entry[1]['canvas_assignment_id']  
        df = pd.DataFrame(columns=['canvas_assignment_id', 'user_param_external_user_id', 'Average Duration'])  
        df = df.append(dict(entry[1]), ignore_index = True) 
        dfs[index] = df  
    else:  
        dfs[index] = dfs[index].append(dict(entry[1]), ignore_index = True)  

#Now page_df and this dictionary dfs should have one to one matches for the loop below
#This loop will find all the student-assignment_id pairs that had an above average duration for that assignment
above_avg_df = pd.DataFrame(columns = ['canvas_assignment_id', 'user_param_external_user_id','Average Duration']) 
for entry in assignment_df.iterrows(): 
    df = dfs[entry[0]] 
    above_avg_chunk = df[df['Average Duration'] >= entry[1][0]] 
    above_avg_df = above_avg_df.append(above_avg_chunk) 
#above_avg_df.to_csv('KSU Students with Above Average Assignment Duration.csv')  

In [116]:
print("Students with longer than average assignmnet durations:")
print(above_avg_df.shape)
print(above_avg_df.head())

Students with longer than average assignmnet durations:
(1018, 3)
   canvas_assignment_id  user_param_external_user_id  Average Duration
0               40876.0                      37827.0      12831.500000
0               40878.0                      37949.0       9549.583333
3               40878.0                      37251.0      10307.500000
0               40879.0                      38399.0      11579.000000
5               40879.0                      40394.0      16064.857143


In [431]:
"""
Data Wrangling Section 2: Course Grades 
Course Grades has been cleand by hand in excel. The details are outlined in Data Wrangling.md
"""
#Import data
df2 = pd.read_csv('/Users/ethanrowe/Dev/springboard_data/KSU Student Final Grades 2018 Anonymous.csv', index_col = 0)
login_times_clean = pd.read_csv('/Users/ethanrowe/Dev/springboard_data/anon_students_login_times_clean.csv', index_col = 0)


In [432]:
#The next step will be to merge login_times_clean and df2. df2 has all of the grade 
#information for certain students by name and id

#I might need to use a different dataset since that one is not anonymous, check Course Final Grades or something later
#login_times_clean is also not anonymous and it has the last access time for students by name and id

#I want a dataframe that has students grades along with last access times. Then I can break that up into those who
#were active before and after certain periods to get a good idea of the distribution.

#All of the students in login_times_clean have final grades less than 70%. I can use df2 for a number of things on its
#own but I also need it to put login_times_clean into perspective.

#First step is to clean df2 if necessary. Then start prepping all of them. They should be EDA ready by the end of this.

"""
To-Do
Load in correct csv's, double check what I need for this
Merge them or something like that or augment login_times_clean with the necessary columns

Do stuff on df2 that is necessary for cleaning like checking for missing values.

Do stuff on login_times_clean such as breaking it up by times and counting stuff. Basically all of this has been 
done in excel already, but this will be more accurate and adaptable moving forward.

"""

,student id,student sis,course,course id,course sis,section,section id,section sis,term,term id,term sis,current score,final score,enrollment state,unposted current score,unposted final score
0,37888,NaN,WELL1000 Foundations for Healthy Living - Howton,622,NaN,WELL 1000/01,1159,NaN,Fall 2018,20,NaN,87.42,86.29,active,87.42,86.29
1,37902,NaN,WELL1000 Foundations for Healthy Living - Howton,622,NaN,WELL 1000/01,1159,NaN,Fall 2018,20,NaN,94.90,93.67,active,94.90,93.67
2,37956,NaN,WELL1000 Foundations for Healthy Living - Howton,622,NaN,WELL 1000/01,1159,NaN,Fall 2018,20,NaN,94.31,93.09,active,94.31,93.09
3,38080,NaN,WELL1000 Foundations for Healthy Living - Howton,622,NaN,WELL 1000/01,1159,NaN,Fall 2018,20,NaN,97.18,95.92,active,97.18,95.92
4,38176,NaN,WELL1000 Foundations for Healthy Living - Howton,622,NaN,WELL 1000/01,1159,NaN,Fall 2018,20,NaN,84.40,83.30,active,84.40,83.30


In [433]:
#df2 is the same as Course Grades (Final Grade Only View) Fall 2018 but with a column for student names
print(df2.head())
print(df2.isna().sum() )
login_times_clean = login_times_clean.sort_index()
print(login_times_clean.head())


   student id  student sis                                            course  \
0       37888          NaN  WELL1000 Foundations for Healthy Living - Howton   
1       37902          NaN  WELL1000 Foundations for Healthy Living - Howton   
2       37956          NaN  WELL1000 Foundations for Healthy Living - Howton   
3       38080          NaN  WELL1000 Foundations for Healthy Living - Howton   
4       38176          NaN  WELL1000 Foundations for Healthy Living - Howton   

   course id  course sis       section  section id  section sis       term  \
0        622         NaN  WELL 1000/01        1159          NaN  Fall 2018   
1        622         NaN  WELL 1000/01        1159          NaN  Fall 2018   
2        622         NaN  WELL 1000/01        1159          NaN  Fall 2018   
3        622         NaN  WELL 1000/01        1159          NaN  Fall 2018   
4        622         NaN  WELL 1000/01        1159          NaN  Fall 2018   

   term id  term sis  current score  final score e

In [434]:
#Theses cell will not run out of order from the rest of the cells since they run permanent changes and use references.

#Let's clean this right now
login_times_clean.columns = ['course id', 'unposted final score',
       'login time', 'student id']

#first convert the login column into something that is readable to pd.to_datetime
for index, entry in enumerate(login_times_clean['login time']):
    login_times_clean['login time'][index] = entry[0:-2] + entry[-2:].upper() + ' 2018'
    if login_times_clean['login time'][index][0:4] == 'Sept':
        login_times_clean['login time'][index] = 'Sep ' + entry[4:]

/Users/ethanrowe/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [435]:
#convert login time new into a datetime object
time_format = '%b %d at %H:%M%p %Y'
login_times_clean['login time'] = pd.to_datetime(login_times_clean['login time'], format=time_format)  

In [377]:
#Now this data frame is prepped for merging
print(login_times_clean.shape)
print(login_times_clean.isna().sum())
login_times_clean.head()


(490, 4)
course id               0
unposted final score    0
login time              0
student id              0
dtype: int64


,course id,unposted final score,login time,student id
0,639,0.0,2018-08-13 10:46:00,38094
1,659,0.0,2018-08-13 11:04:00,22168
2,659,0.0,2018-08-13 03:19:00,37144
3,638,0.0,2018-08-13 09:04:00,38000
4,631,0.0,2018-08-14 01:12:00,38410


In [437]:
#Merge
grades_login_times = pd.merge(login_times_clean, df2, on = ['student id', 'course id', 'unposted final score'], how = 'left')

#Now this is EDA ready

print(grades_login_times.isna().sum())
grades_login_times
#grades_login_times.to_csv('grades_and_login_times(below 70).csv')

#why are there so many NaN values even though each dataframe has no missing values and I think they originally came 
#from the same dataframe...?
#Shit it looks like there are student ids that are not in df2 along with student ids that do not match their course id
#in the dataframe, have to figure out who those students are or something. I don't want to deal with that shit right
#now though. I should do some EDA on the other dataframe and then come back.
#To check maybe I should write code to see if all the student ids from this side are in the other one too

course id                   0
unposted final score        0
login time                  0
student id                  0
student sis               490
course                      0
course sis                490
section                     0
section id                  0
section sis               490
term                        0
term id                     0
term sis                  490
current score               0
final score                 0
enrollment state            0
unposted current score      0
dtype: int64


,course id,unposted final score,login time,student id,student sis,course,course sis,section,section id,section sis,term,term id,term sis,current score,final score,enrollment state,unposted current score
0,639,0.00,2018-08-13 10:46:00,38094,NaN,WELL1000 Foundations for Healthy Living - Onli...,NaN,WELL 1000/W72,1191,NaN,Fall 2018,20,NaN,0.00,0.00,active,0.00
1,659,0.00,2018-08-13 11:04:00,22168,NaN,WELL1000 Foundations for Healthy Living - Onli...,NaN,WELL 1000/W87,1230,NaN,Fall 2018,20,NaN,0.00,0.00,concluded,0.00
2,659,0.00,2018-08-13 03:19:00,37144,NaN,WELL1000 Foundations for Healthy Living - Onli...,NaN,WELL 1000/W87,1230,NaN,Fall 2018,20,NaN,0.00,0.00,concluded,0.00
3,638,0.00,2018-08-13 09:04:00,38000,NaN,WELL1000 Foundations for Healthy Living - Onli...,NaN,WELL 1000/W75,1190,NaN,Fall 2018,20,NaN,0.00,0.00,active,0.00
4,631,0.00,2018-08-14 01:12:00,38410,NaN,WELL1000 Foundations for Healthy Living - Onli...,NaN,WELL 1000/W63,1176,NaN,Fall 2018,20,NaN,0.00,0.00,active,0.00
5,633,6.21,2018-08-14 01:45:00,38133,NaN,WELL1000 Foundations for Healthy Living - Onli...,NaN,WELL 1000/W65,1179,NaN,Fall 2018,20,NaN,6.51,6.21,active,6.51
6,644,0.00,2018-08-14 11:12:00,38304,NaN,WELL1000 Foundations for Healthy Living - Mao,NaN,WELL 1000/04,1201,NaN,Fall 2018,20,NaN,0.00,0.00,concluded,0.00
7,659,0.00,2018-08-14 11:29:00,38320,NaN,WELL1000 Foundations for Healthy Living - Onli...,NaN,WELL 1000/W87,1230,NaN,Fall 2018,20,NaN,0.00,0.00,concluded,0.00
8,639,1.69,2018-08-14 11:49:00,38903,NaN,WELL1000 Foundations for Healthy Living - Onli...,NaN,WELL 1000/W72,1191,NaN,Fall 2018,20,NaN,1.69,1.69,active,1.69
9,636,0.00,2018-08-14 12:12:00,37677,NaN,WELL1000 Foundations for Healthy Living - Onli...,NaN,WELL 1000/W68,1185,NaN,Fall 2018,20,NaN,0.00,0.00,active,0.00


In [438]:
#Find the students who stopped being active before the drop date Aug 19
drop_date = pd.Timestamp(datetime.date(2018, 8, 20))
before_aug = grades_login_times[grades_login_times['login time'] <= drop_date]
print(before_aug.shape)

#Find the students who stopped being before the other date thing, Oct 3
other_date = pd.Timestamp(datetime.date(2018, 10, 4))
before_oct = grades_login_times[(grades_login_times['login time'] >= drop_date) & (grades_login_times['login time'] <= other_date)]
print(before_oct.shape)


(45, 17)
(119, 17)


In [120]:
"""
Other Data sets to consider are Click History and Assessment Responses
"""

'\nOther Data sets to consider are Click History and Assessment Responses\n'